In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import librosa
import sklearn
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from matplotlib import style
import pandas
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import csv
from sklearn import svm
import itertools
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import pandas as pd
from scipy.special import comb
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif, mutual_info_classif, SelectPercentile, GenericUnivariateSelect
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from scipy.stats import rankdata

In [2]:
DATADIR = "/Users/yash_/Documents/project/"
CATEGORIES = ["Segmented_Laugh", "Segmented_NonLaugh"]

# Defining Functions

In [4]:
def createcombinations(n):
    elements = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
    a = list(itertools.combinations(elements, n))
    a = np.asarray(a).astype(int)
    return a

In [5]:
def create_training_data(t_frame, t_shift, n_mfccs, component, training_data, n_features):
    count=0
    error_count = 0
    laugh_counter = 0
    nonlaugh_counter = 0
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category) 
        class_num = CATEGORIES.index(category)
        for aud in os.listdir(path):
            if aud == '.DS_Store':
                continue
            
            aud_array , sr = librosa.load(os.path.join(path,aud), sr=None)
            count+=1

            mfccs = []
            
            try:
                mfcc = (librosa.feature.mfcc(aud_array, sr=sr,  n_mfcc=20,  win_length = int(sr*t_frame), hop_length = int(sr*t_shift))) 
                mfcc_temp = mfcc[component,:]
                #mfcc_temp = librosa.feature.delta(mfcc_temp)
                #mfcc_temp = librosa.feature.delta(mfcc_temp)
                mean_mfccs = np.mean(np.asarray(mfcc_temp),axis = 1)
                #var_mfccs = np.var(np.asarray(mfcc_temp), axis = 1)
                
                mfccs.append(mean_mfccs)
                #mfccs.append(var_mfccs)
                mfccs = np.asarray(mfccs).reshape(n_mfccs*n_features,1)
                
                training_data.append([mfccs.reshape(-1,1), class_num])
                if category == 'Segmented_Laugh':
                    laugh_counter +=1
                else:
                    nonlaugh_counter += 1
                    if nonlaugh_counter >= laugh_counter:
                        break

            except ValueError:
                pass
            
    return laugh_counter, nonlaugh_counter
    

# Main Loop

In [7]:
def scaling(X,n_mfcc):
    for i in range(n_mfcc):
        X[:,i] = X[:,i]/(np.mean(X[:,i]))
        

    #X[:,1] = np.square(X[:,1])
        
    return X

In [ ]:
n_features = 1
t_frame = 0.025
t_shift = 0.01
n_mfcc = 20
component = createcombinations(n_mfcc)

for components in component:
    training_data = []
    laugh, nonlaugh = create_training_data(t_frame, t_shift, n_mfcc, components, training_data, n_features)
    print(laugh, nonlaugh)
    
random.shuffle(training_data) 
X = []
Y = []
for features, label in training_data:
    X.append(features)
    Y.append(label)
        
X = np.array(X).reshape(-1,n_mfcc*n_features)
Y = np.array(Y)
X = scaling(X,n_mfcc)

X_train, X_test, y_train, y_test = train_test_split(
         X, Y, test_size=0.2, random_state=123)

# SFS ( if forward = True ) and SBS ( if forward = False)

#we will try knn, Gmm, Rf and SVM

In [ ]:

knn = KNeighborsClassifier(n_neighbors=2) # ml_algo used = knn
sfs1 = SFS(knn, 
           k_features=3, 
           forward=True, # if forward = True then SFS otherwise SBS
           floating=False, 
           verbose=2,
           scoring='accuracy'
           )

pipe = Pipeline([('sfs', sfs1), 
                 ('knn', knn)])

param_grid = [
  {'sfs__k_features': [2,4,6,8,10],
   'sfs__estimator__n_neighbors': [1, 5, 10,15,17,19]}
  ] # dict. for tuning inside and outside hyperparameters i.e. no. of k_features and no. of neighbors( for knn ) 

gs = GridSearchCV(estimator=pipe, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  n_jobs=1, 
                  cv=5,
                  iid=True,
                  refit=False)

# run gridearch
gs = gs.fit(X_train, y_train)

# Saving the results in a Dataframe

In [ ]:
index_0 = []
index_1 = []
result = []
for i in range(len(gs.cv_results_['params'])):
    dic = gs.cv_results_['params'][i].values()
    lis = list(dic)
    index_0.append(int(lis[0]))
    result.append(float(gs.cv_results_['mean_test_score'][i]))
    index_1.append(int(lis[1]))

    
dataframe  = pd.DataFrame(columns=['sfs__estimator__n_neighbors','sfs__k_features','Test Accuracy'])
dataframe['sfs__estimator__n_neighbors'] = np.array(index_0) 
#change the name of 0th column for different algos. i.e ex. for RF name = 'sfs__estimator__n_trees'
dataframe['sfs__k_features'] = np.array(index_1)
dataframe['Test Accuracy'] = np.array(result)

#showing the parameters with best results

In [ ]:
pipe.set_params(**gs.best_params_).fit(X_train, y_train)
print('Best features:', pipe.steps[0][1].k_feature_idx_)

print('Best score:', gs.best_score_)

# Saving the dataframe into CSV file

In [ ]:
dataframe.to_csv('/Users/yash_/Documents/project/file1.csv')
# change the address accordingly

# Plot for a particular k_feature (run_after_tuning)

In [ ]:
style.use('dark_background')
fig1 = plot_sfs(sfs1.get_metric_dict(), kind='std_dev')
plt.title('Sequential Backward Selection (w. StdDev)')
#plt.grid()
plt.show()